In [50]:
from pathlib import Path 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from collections import OrderedDict
import sys
import os
import seaborn as sns
import researchpy as rp
import statsmodels.formula.api as smf
import scipy.stats as stats
import ipympl

#sys.path.append('/Users/alina/Desktop/MIT/code/ADHD/MTA/helper')
from helper import rr, prep, var_dict

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
if Path('/Volumes/Samsung_T5/MIT/mta').exists():
    data_root =     '/Volumes/Samsung_T5/MIT/mta'
    data_derived = '/Volumes/Samsung_T5/MIT/mta/output/derived_data'
else: 
    data_root = '/Users/alina/Desktop/MIT/code/data'
    data_derived= '/Users/alina/Desktop/MIT/code/data/output/derived_data'

In [52]:
os.listdir(data_derived)

['ssrs_b_mediators.csv',
 'accept.csv',
 'masc_24_mediators.csv',
 'outcome_means_b_14.csv',
 '.DS_Store',
 'snap_b_mediators.csv',
 'ssrs_24_mediators.csv',
 'original_and_new_rr_14.csv',
 'original_and_new_rr_14_contrasts.csv',
 'snap_36_mediators.csv',
 'pc_b_mediators.csv',
 'masc_14_mediators.csv',
 'ssrs_14_mediators.csv',
 'comorb.csv',
 'pc_36_mediators.csv',
 'masc_14_and_mediators.csv',
 'assist.csv',
 'pc_24_mediators.csv',
 'original_rr_results_14.csv',
 'snap_24_mediators.csv',
 'ssrs_14_and_mediators.csv',
 'ssrs_36_mediators.csv',
 'prev_med.csv',
 'pc_14_mediators.csv',
 'pc_14_and_mediators.csv',
 'snap_14_mediators.csv',
 'snap_14_and_mediators.csv',
 'masc_36_mediators.csv',
 'masc_b_mediators.csv',
 'treatment_groups.csv']

### Read data 

In [53]:

baseline_var = ['src_subject_id', 'interview_date', 'interview_age', 'sex', 'site', 'days_baseline']
dtypes_baseline = { 'src_subject_id' : 'str',
                    'interview_date': 'str' , 
                    'interview_age' : 'int64' ,
                    'sex' : 'str', 
                    'site' : 'int64' ,
                    'days_baseline':  'int64',
                    'version_form': 'str'}

version_form = ['version_form']

qsts = ['snap', 'ssrs',  'pc','wechsler', 'masc']##, 'wechsler'] #masc to many missing data 


snap_file = 'snap01.txt'
ssrs_file = 'ssrs01.txt'
masc_file = 'masc_p01.txt'
parent_child_file = 'pcrc01.txt'
wechsler_file = 'wiat_iiip201.txt'
treat_group_file = 'treatment_groups.csv'
#outcome variablles 
snap_vars = ['snainatx', 'snahix', 'snaoddx'] #inattention_mean, hyperactie mean
ssrs_vars = ['sspintx', 'ssptossx']# social skills mean, internalizing mean 
masc_vars = ['masc_masctotalt']
pc_vars = ['pcrcpax', 'pcrcprx'] # power assertion, personal closeness
wechsler_vars = ['w1readb','w2math','w3spell' ]

outcomes_dict  = {'snap' : snap_vars, 'ssrs' : ssrs_vars,  "masc": masc_vars, 'pc': pc_vars, 'wechsler': wechsler_vars}

interaction_predictors = ['days_baseline', 'site', 'trtname'] #time, site, treatment group

# mediator variables
# comorb_mediators  = ['cdorodd' , 'pso', 'psoi', 'pag', 'pagi', 'pga', 'pgai' ,'psa'] #ODD/CD or anx excluding specific phobia 
# services_mediators =  ['demo61'] #reciept of public assistance 
# prev_med_mediators = ['hi_24'] #medication intake prior to study 

# #moderator variables 
# accept_moderator = ['d2dresp'] # initail acceptance of treatment 
# raters = ['Teacher', 'Parent']

treat_group = pd.read_csv(Path(data_derived, treat_group_file))


snap = prep.get_data(Path(data_root, snap_file), columns= [baseline_var, snap_vars, version_form], treat_group= treat_group, set_dtypes= True, version_form= True, split_timepoints= True)
ssrs = prep.get_data(Path(data_root, ssrs_file), columns= [baseline_var, ssrs_vars, version_form], treat_group= treat_group, set_dtypes= True, version_form= True, split_timepoints= True)
pc = prep.get_data(Path(data_root, parent_child_file), columns= [baseline_var, pc_vars], treat_group= treat_group, set_dtypes= True, version_form= False, split_timepoints= True)
masc = prep.get_data(Path(data_root,masc_file), columns= [baseline_var, masc_vars], treat_group= treat_group, set_dtypes= True, version_form= False, split_timepoints= True)
wechsler = prep.get_data(Path(data_root, wechsler_file), columns= [baseline_var, wechsler_vars], treat_group= treat_group, set_dtypes= True, version_form= False, split_timepoints= True)
data_dict = dict(zip(qsts, [snap, ssrs, masc, pc, wechsler]))

for key in data_dict.keys():
    for time, df in data_dict[key].items():

        data_dict[key][time] = data_dict[key][time][data_dict[key][time]['trtname'] != 'L']

                                  

Success
No timepoints specifed. Using (46,168,319,500) by default.
Success
No timepoints specifed. Using (46,168,319,500) by default.
Success
No timepoints specifed. Using (46,168,319,500) by default.
Success
No timepoints specifed. Using (46,168,319,500) by default.
Success
No timepoints specifed. Using (46,168,319,500) by default.


### parameters 

In [54]:
hyps_interactions = rr.get_hyps_interactions()
alpha = 0.05 
group = 'src_subject_id'
hyps_interactions

{'site': 'C(site)[T.2] = C(site)[T.3] = C(site)[T.4] = C(site)[T.5] = C(site)[T.6] = 0',
 'time': 'days_baseline = 0',
 'treat': 'C(trtname, Treatment(reference="A"))[T.M] = C(trtname, Treatment(reference="A"))[T.P] = C(trtname, Treatment(reference="A"))[T.C] = 0',
 'site_treat': 'C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.2] = C(trtname, Treatment(reference="A"))[T.P]:C(site)[T.2] = C(trtname, Treatment(reference="A"))[T.C]:C(site)[T.2] = C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.3] = C(trtname, Treatment(reference="A"))[T.P]:C(site)[T.3] = C(trtname, Treatment(reference="A"))[T.C]:C(site)[T.3] = C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.4] = C(trtname, Treatment(reference="A"))[T.P]:C(site)[T.4] = C(trtname, Treatment(reference="A"))[T.C]:C(site)[T.4] = C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.5] = C(trtname, Treatment(reference="A"))[T.P]:C(site)[T.5] = C(trtname, Treatment(reference="A"))[T.C]:C(site)[T.5] = C(trtname, Treatment(reference="A"

In [43]:
all_predictors = [ 'treat',  'time', 'site']

snap_outcome = ['snainatx', 'snahix', 'snaoddx']
snap_var = ['snap_inatt', 'snap_hyp', 'snap_imp', 'snap_odd']

ssrs_outcome = ['ssptossx','sspintx' ]
ssrs_var = ['ssrs_ss', 'ssrs_int']

masc_outcome =  ['masc_masctotalt']
masc_var = ['masc_tot']

wechsler_outcome = ['w1readb', 'w2math', 'w3spell']
wechsler_var = ['w_read', 'w_math', 'w_spell']

outcome_all = np.concatenate((snap_outcome, ssrs_outcome, masc_outcome, wechsler_outcome))
vars_all = np.concatenate((snap_var, ssrs_var, masc_var, wechsler_var))

dict_outcome_var = dict(zip(vars_all, outcome_all))
dict_outcome_var


{'snap_inatt': 'snainatx',
 'snap_hyp': 'snahix',
 'snap_imp': 'snaoddx',
 'snap_odd': 'ssptossx',
 'ssrs_ss': 'sspintx',
 'ssrs_int': 'masc_masctotalt',
 'masc_tot': 'w1readb',
 'w_read': 'w2math',
 'w_math': 'w3spell'}

In [44]:
raters = ['Teacher', 'Parent']
times = ['b', '14', '24', '36']

In [45]:
data_dict

{'snap': {'b':       src_subject_id interview_date  interview_age sex  snainatx  snaoddx  \
  0              P1001     1997-06-28            156   M      0.11     0.00   
  1              P1001     1997-06-28            156   M      0.33     0.00   
  6              P1002     1997-05-02            155   M      2.00     1.63   
  7              P1002     1997-06-03            120   M      1.00     1.25   
  8              P1002     1997-06-03            120   M      1.11     0.88   
  ...              ...            ...            ...  ..       ...      ...   
  14513          P1864     1998-05-12             96   M      0.11     0.00   
  14517          P1865     1998-03-18             96   M      0.56     0.38   
  14522          P1866     1998-07-17             96   F      0.11     0.00   
  14528          P1867     1998-01-28             96   M      0.78     0.25   
  14536          P1868     1998-01-30             96   F      1.00     0.13   
  
         snahix  days_baseline site 

In [46]:
data = data_dict['snap']['14']
rater = 'Teacher'
out = 'snahix'
alpha = 0.05
groups = 'src_subject_id'
data[groups]


0        P1001
1        P1001
6        P1002
7        P1002
8        P1002
         ...  
14513    P1864
14517    P1865
14522    P1866
14528    P1867
14536    P1868
Name: src_subject_id, Length: 4278, dtype: object

In [47]:
data

,src_subject_id,interview_date,interview_age,sex,snainatx,snaoddx,snahix,days_baseline,site,version_form,trtname
0,P1001,1997-06-28,156,M,0.11,0.00,0.22,0,1,Parent,L
1,P1001,1997-06-28,156,M,0.33,0.00,0.00,0,1,Parent,L
6,P1002,1997-05-02,155,M,2.00,1.63,1.56,-32,1,Parent,C
7,P1002,1997-06-03,120,M,1.00,1.25,0.78,0,1,Parent,C
8,P1002,1997-06-03,120,M,1.11,0.88,1.00,0,1,Parent,C
...,...,...,...,...,...,...,...,...,...,...,...
14513,P1864,1998-05-12,96,M,0.11,0.00,0.33,0,5,Teacher,L
14517,P1865,1998-03-18,96,M,0.56,0.38,0.11,0,3,Teacher,L
14522,P1866,1998-07-17,96,F,0.11,0.00,0.00,0,6,Teacher,L
14528,P1867,1998-01-28,96,M,0.78,0.25,0.33,0,4,Teacher,L


In [48]:

test = rr.get_RR_stats(formula= formulas['snahix'], data = data[data['version_form'] == rater],  groups = 'src_subject_id',  alpha= alpha)

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: 

In [55]:
formulas = dict(zip( outcome_all, [rr.get_formula(outcome, all_predictors) for outcome in outcome_all]))
formulas

{'snainatx': 'snainatx ~ C(trtname, Treatment(reference="L")) * days_baseline * C(site)',
 'snahix': 'snahix ~ C(trtname, Treatment(reference="L")) * days_baseline * C(site)',
 'snaoddx': 'snaoddx ~ C(trtname, Treatment(reference="L")) * days_baseline * C(site)',
 'ssptossx': 'ssptossx ~ C(trtname, Treatment(reference="L")) * days_baseline * C(site)',
 'sspintx': 'sspintx ~ C(trtname, Treatment(reference="L")) * days_baseline * C(site)',
 'masc_masctotalt': 'masc_masctotalt ~ C(trtname, Treatment(reference="L")) * days_baseline * C(site)',
 'w1readb': 'w1readb ~ C(trtname, Treatment(reference="L")) * days_baseline * C(site)',
 'w2math': 'w2math ~ C(trtname, Treatment(reference="L")) * days_baseline * C(site)',
 'w3spell': 'w3spell ~ C(trtname, Treatment(reference="L")) * days_baseline * C(site)'}

In [57]:
rr_dict , p_vals_dict , f_val_dict = {}, {}, {}
rr_list = []

time = times[1] 
for qst in data_dict.keys():
    data = data_dict[qst][time]
    
    #data= data[data['trtname'] != 'L']
    print(data.trtname.value_counts())

    outcomes = outcomes_dict[qst]
    print(outcomes)
    version_form = raters if qst == 'snap' or qst == 'ssrs'  else None
    contrasts =[]

    print(qst, data.shape, outcomes, version_form, alpha)
    rr_results = []
    if version_form is not None:
        for rater in raters:
            for out in outcomes: 
                print(out, rater)
                rr_results.append(rr.get_RR_stats(formulas[out], data[data['version_form'] == rater], groups = 'src_subject_id',  alpha= alpha))
    else :

        for out in outcomes:
            print(out)
            rr_results.append(rr.get_RR_stats(formulas[out], data, groups = 'src_subject_id', alpha= alpha) )

            

    rr_dict[qst] = rr_results
    rr_list.append(rr_results)

trtname
C    907
P    893
A    874
M    871
L      0
Name: count, dtype: int64
['snainatx', 'snahix', 'snaoddx']
snap (3545, 11) ['snainatx', 'snahix', 'snaoddx'] ['Teacher', 'Parent'] 0.05
snainatx Teacher


LinAlgError: Singular matrix

{'snainatx': 'snainatx ~ C(trtname, Treatment(reference="L")) * days_baseline * C(site)',
 'snahix': 'snahix ~ C(trtname, Treatment(reference="L")) * days_baseline * C(site)',
 'snaoddx': 'snaoddx ~ C(trtname, Treatment(reference="L")) * days_baseline * C(site)',
 'ssptossx': 'ssptossx ~ C(trtname, Treatment(reference="L")) * days_baseline * C(site)',
 'sspintx': 'sspintx ~ C(trtname, Treatment(reference="L")) * days_baseline * C(site)',
 'masc_masctotalt': 'masc_masctotalt ~ C(trtname, Treatment(reference="L")) * days_baseline * C(site)',
 'w1readb': 'w1readb ~ C(trtname, Treatment(reference="L")) * days_baseline * C(site)',
 'w2math': 'w2math ~ C(trtname, Treatment(reference="L")) * days_baseline * C(site)',
 'w3spell': 'w3spell ~ C(trtname, Treatment(reference="L")) * days_baseline * C(site)'}

## SNAP - inattention 

In [15]:
outcome = outcome_all[0]
outcome

'snainatx'

### Parent and teacher 

In [15]:
res_snap_14_p, summ_snap_14_p , h = rr.get_RR_stats(formulas[outcome] , snap14_p, group, alpha)
res_snap_14_t, summ_snap_14_t , h2 = rr.get_RR_stats(formulas[outcome], snap14_t, group,  alpha)
f_test_snap14_p = rr.f_test_interactions(res_snap_14_p, hyps_interactions, alpha)
f_test_snap14_t = rr.f_test_interactions(res_snap_14_t, hyps_interactions, alpha)

f_test_snap14 = pd.concat([f_test_snap14_p, f_test_snap14_p], axis=1)
f_test_snap14


/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value,Description,Significance,F-Value,P-Value
0,site,Not Significant,1.230084,2.922964e-01,site,Not Significant,1.230084,2.922964e-01
1,time,*Significant*,4.110933,4.273535e-02,time,*Significant*,4.110933,4.273535e-02
2,treat,*Significant*,17.428887,3.562197e-11,treat,*Significant*,17.428887,3.562197e-11
3,site_treat,Not Significant,1.168429,2.894533e-01,site_treat,Not Significant,1.168429,2.894533e-01
4,time_treat,*Significant*,3.284458,2.004706e-02,time_treat,*Significant*,3.284458,2.004706e-02
5,site_time_treat,Not Significant,0.988149,4.546650e-01,site_time_treat,Not Significant,0.988149,4.546650e-01


In [16]:
res_snap_24_p, summ_snap_24_p , h24 = rr.get_RR_stats(formulas[outcome] , snap24_p, group, alpha)
res_snap_24_t, summ_snap_24_t , h242 = rr.get_RR_stats(formulas[outcome] , snap24_t, group,  alpha)
f_test_snap24_p  = rr.f_test_interactions(res_snap_24_p, hyps_interactions, alpha)
f_test_snap24_t  = rr.f_test_interactions(res_snap_24_t, hyps_interactions, alpha)

f_test_snap24 = pd.concat([f_test_snap24_p, f_test_snap24_p], axis=1)
f_test_snap24

NameError: name 'rr_helper' is not defined

In [ ]:
res_snap_36_p, summ_snap_36_p , h36 = rr.get_RR_stats(formulas[outcome] , snap36_p, group, alpha)
res_snap_36_t, summ_snap_36_t , h362 = rr.get_RR_stats(formulas[outcome] , snap36_t, group,  alpha)
f_test_snap36_p  = rr.f_test_interactions(res_snap_36_p, hyps_interactions, alpha)
f_test_snap36_t  = rr.f_test_interactions(res_snap_36_t, hyps_interactions, alpha)

f_test_snap36 = pd.concat([f_test_snap36_p, f_test_snap36_p], axis=1)
f_test_snap36

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value,Description,Significance,F-Value,P-Value
0,site,Not Significant,0.917978,4.681338e-01,site,Not Significant,0.917978,4.681338e-01
1,time,Not Significant,2.949261,8.603695e-02,time,Not Significant,2.949261,8.603695e-02
2,treat,*Significant*,25.104450,5.249829e-16,treat,*Significant*,25.104450,5.249829e-16
3,site_treat,Not Significant,1.284443,2.029175e-01,site_treat,Not Significant,1.284443,2.029175e-01
4,time_treat,*Significant*,5.056892,1.709547e-03,time_treat,*Significant*,5.056892,1.709547e-03
5,site_time_treat,Not Significant,0.847160,5.925485e-01,site_time_treat,Not Significant,0.847160,5.925485e-01


### SNAP Hyperctivity 

In [ ]:
outcome = dict_outcome_var['snap_hyp']
outcome

'snahypax'

In [ ]:
res_snap_14_p, summ_snap_14_p , h = rr.get_RR_stats(formulas[outcome] , snap14_p, group, alpha)
res_snap_14_t, summ_snap_14_t , h2 = rr.get_RR_stats(formulas[outcome], snap14_t, group,  alpha)
f_test_snap14_p = rr.f_test_interactions(res_snap_14_p, hyps_interactions, alpha)
f_test_snap14_t = rr.f_test_interactions(res_snap_14_t, hyps_interactions, alpha)

f_test_snap14 = pd.concat([f_test_snap14_p, f_test_snap14_p], axis=1)
f_test_snap14

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value,Description,Significance,F-Value,P-Value
0,site,Not Significant,0.361466,8.750420e-01,site,Not Significant,0.361466,8.750420e-01
1,time,Not Significant,2.410911,1.206458e-01,time,Not Significant,2.410911,1.206458e-01
2,treat,*Significant*,27.339719,2.460142e-17,treat,*Significant*,27.339719,2.460142e-17
3,site_treat,Not Significant,1.585151,7.008391e-02,site_treat,Not Significant,1.585151,7.008391e-02
4,time_treat,*Significant*,6.696426,1.700017e-04,time_treat,*Significant*,6.696426,1.700017e-04
5,site_time_treat,Not Significant,0.704308,7.355319e-01,site_time_treat,Not Significant,0.704308,7.355319e-01


In [ ]:
res_snap_24_p, summ_snap_24_p , h24 = rr.get_RR_stats(formulas[outcome] , snap24_p, group, alpha)
res_snap_24_t, summ_snap_24_t , h242 = rr.get_RR_stats(formulas[outcome] , snap24_t, group,  alpha)
f_test_snap24_p  = rr.f_test_interactions(res_snap_24_p, hyps_interactions, alpha)
f_test_snap24_t  = rr.f_test_interactions(res_snap_24_t, hyps_interactions, alpha)

f_test_snap24 = pd.concat([f_test_snap24_p, f_test_snap24_p], axis=1)
f_test_snap24

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value,Description,Significance,F-Value,P-Value
0,site,Not Significant,0.396190,8.517167e-01,site,Not Significant,0.396190,8.517167e-01
1,time,Not Significant,2.370746,1.237594e-01,time,Not Significant,2.370746,1.237594e-01
2,treat,*Significant*,32.768002,9.374779e-21,treat,*Significant*,32.768002,9.374779e-21
3,site_treat,Not Significant,1.035750,4.140879e-01,site_treat,Not Significant,1.035750,4.140879e-01
4,time_treat,*Significant*,7.636891,4.430050e-05,time_treat,*Significant*,7.636891,4.430050e-05
5,site_time_treat,Not Significant,0.559079,8.629284e-01,site_time_treat,Not Significant,0.559079,8.629284e-01


In [ ]:
res_snap_36_p, summ_snap_36_p , h36 = rr.get_RR_stats(formulas[outcome] , snap36_p, group, alpha)
res_snap_36_t, summ_snap_36_t , h362 = rr.get_RR_stats(formulas[outcome] , snap36_t, group,  alpha)
f_test_snap36_p  = rr.f_test_interactions(res_snap_36_p, hyps_interactions, alpha)
f_test_snap36_t  = rr.f_test_interactions(res_snap_36_t, hyps_interactions, alpha)

f_test_snap36 = pd.concat([f_test_snap36_p, f_test_snap36_p], axis=1)
f_test_snap36

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value,Description,Significance,F-Value,P-Value
0,site,Not Significant,0.325295,8.979766e-01,site,Not Significant,0.325295,8.979766e-01
1,time,Not Significant,2.271275,1.319139e-01,time,Not Significant,2.271275,1.319139e-01
2,treat,*Significant*,34.198817,1.139406e-21,treat,*Significant*,34.198817,1.139406e-21
3,site_treat,Not Significant,0.938866,5.194677e-01,site_treat,Not Significant,0.938866,5.194677e-01
4,time_treat,*Significant*,7.281001,7.334639e-05,time_treat,*Significant*,7.281001,7.334639e-05
5,site_time_treat,Not Significant,0.712115,7.280012e-01,site_time_treat,Not Significant,0.712115,7.280012e-01


### SNAP impulsivity 

In [ ]:
outcome = dict_outcome_var['snap_imp']
outcome

'snaimpux'

In [ ]:
res_snap_14_p, summ_snap_14_p , h = rr.get_RR_stats(formulas[outcome] , snap14_p, group, alpha)
res_snap_14_t, summ_snap_14_t , h2 = rr.get_RR_stats(formulas[outcome], snap14_t, group,  alpha)
f_test_snap14_p = rr.f_test_interactions(res_snap_14_p, hyps_interactions, alpha)
f_test_snap14_t = rr.f_test_interactions(res_snap_14_t, hyps_interactions, alpha)

f_test_snap14 = pd.concat([f_test_snap14_p, f_test_snap14_p], axis=1)
f_test_snap14

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value,Description,Significance,F-Value,P-Value
0,site,Not Significant,0.481489,7.903134e-01,site,Not Significant,0.481489,7.903134e-01
1,time,Not Significant,1.288670,2.564252e-01,time,Not Significant,1.288670,2.564252e-01
2,treat,*Significant*,17.877325,1.871069e-11,treat,*Significant*,17.877325,1.871069e-11
3,site_treat,Not Significant,1.594423,6.762378e-02,site_treat,Not Significant,1.594423,6.762378e-02
4,time_treat,*Significant*,2.831360,3.708864e-02,time_treat,*Significant*,2.831360,3.708864e-02
5,site_time_treat,*Significant*,2.009524,2.410370e-02,site_time_treat,*Significant*,2.009524,2.410370e-02


In [ ]:
res_snap_24_p, summ_snap_24_p , h24 = rr.get_RR_stats(formulas[outcome] , snap24_p, group, alpha)
res_snap_24_t, summ_snap_24_t , h242 = rr.get_RR_stats(formulas[outcome] , snap24_t, group,  alpha)
f_test_snap24_p  = rr.f_test_interactions(res_snap_24_p, hyps_interactions, alpha)
f_test_snap24_t  = rr.f_test_interactions(res_snap_24_t, hyps_interactions, alpha)

f_test_snap24 = pd.concat([f_test_snap24_p, f_test_snap24_p], axis=1)
f_test_snap24

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value,Description,Significance,F-Value,P-Value
0,site,Not Significant,0.332107,8.937767e-01,site,Not Significant,0.332107,8.937767e-01
1,time,Not Significant,0.938504,3.327602e-01,time,Not Significant,0.938504,3.327602e-01
2,treat,*Significant*,22.040078,4.452643e-14,treat,*Significant*,22.040078,4.452643e-14
3,site_treat,Not Significant,1.046493,4.030418e-01,site_treat,Not Significant,1.046493,4.030418e-01
4,time_treat,*Significant*,3.881111,8.814474e-03,time_treat,*Significant*,3.881111,8.814474e-03
5,site_time_treat,*Significant*,1.895047,3.556340e-02,site_time_treat,*Significant*,1.895047,3.556340e-02


In [ ]:
res_snap_36_p, summ_snap_36_p , h36 = rr.get_RR_stats(formulas[outcome] , snap36_p, group, alpha)
res_snap_36_t, summ_snap_36_t , h362 = rr.get_RR_stats(formulas[outcome] , snap36_t, group,  alpha)
f_test_snap36_p  = rr.f_test_interactions(res_snap_36_p, hyps_interactions, alpha)
f_test_snap36_t  = rr.f_test_interactions(res_snap_36_t, hyps_interactions, alpha)

f_test_snap36 = pd.concat([f_test_snap36_p, f_test_snap36_p], axis=1)
f_test_snap36

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value,Description,Significance,F-Value,P-Value
0,site,Not Significant,0.306887,9.090112e-01,site,Not Significant,0.306887,9.090112e-01
1,time,Not Significant,0.907971,3.407428e-01,time,Not Significant,0.907971,3.407428e-01
2,treat,*Significant*,21.901683,5.284184e-14,treat,*Significant*,21.901683,5.284184e-14
3,site_treat,Not Significant,1.041469,4.081502e-01,site_treat,Not Significant,1.041469,4.081502e-01
4,time_treat,*Significant*,3.329234,1.881997e-02,time_treat,*Significant*,3.329234,1.881997e-02
5,site_time_treat,Not Significant,1.599444,9.224643e-02,site_time_treat,Not Significant,1.599444,9.224643e-02


### SNAP ODD 

In [ ]:
outcome = dict_outcome_var['snap_odd']
outcome

'snaoddx'

In [ ]:
res_snap_14_p, summ_snap_14_p , h = rr.get_RR_stats(formulas[outcome] , snap14_p, group, alpha)
res_snap_14_t, summ_snap_14_t , h2 = rr.get_RR_stats(formulas[outcome], snap14_t, group,  alpha)
f_test_snap14_p = rr.f_test_interactions(res_snap_14_p, hyps_interactions, alpha)
f_test_snap14_t = rr.f_test_interactions(res_snap_14_t, hyps_interactions, alpha)

f_test_snap14 = pd.concat([f_test_snap14_p, f_test_snap14_p], axis=1)
f_test_snap14

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value,Description,Significance,F-Value,P-Value
0,site,Not Significant,0.415264,0.838422,site,Not Significant,0.415264,0.838422
1,time,Not Significant,3.491899,0.061812,time,Not Significant,3.491899,0.061812
2,treat,*Significant*,9.229401,0.000005,treat,*Significant*,9.229401,0.000005
3,site_treat,*Significant*,1.679549,0.048364,site_treat,*Significant*,1.679549,0.048364
4,time_treat,Not Significant,1.797468,0.145577,time_treat,Not Significant,1.797468,0.145577
5,site_time_treat,Not Significant,1.621268,0.086438,site_time_treat,Not Significant,1.621268,0.086438


In [ ]:
res_snap_24_p, summ_snap_24_p , h24 = rr.get_RR_stats(formulas[outcome] , snap24_p, group, alpha)
res_snap_24_t, summ_snap_24_t , h242 = rr.get_RR_stats(formulas[outcome] , snap24_t, group,  alpha)
f_test_snap24_p  = rr.f_test_interactions(res_snap_24_p, hyps_interactions, alpha)
f_test_snap24_t  = rr.f_test_interactions(res_snap_24_t, hyps_interactions, alpha)

f_test_snap24 = pd.concat([f_test_snap24_p, f_test_snap24_p], axis=1)
type(f_test_snap24)

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


pandas.core.frame.DataFrame

In [ ]:
res_snap_36_p, summ_snap_36_p , h36 = rr.get_RR_stats(formulas[outcome] , snap36_p, group, alpha)
res_snap_36_t, summ_snap_36_t , h362 = rr.get_RR_stats(formulas[outcome] , snap36_t, group,  alpha)
f_test_snap36_p  = rr.f_test_interactions(res_snap_36_p, hyps_interactions, alpha)
f_test_snap36_t  = rr.f_test_interactions(res_snap_36_t, hyps_interactions, alpha)

f_test_snap36 = pd.concat([f_test_snap36_p, f_test_snap36_p], axis=1)
f_test_snap36

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value,Description,Significance,F-Value,P-Value
0,site,Not Significant,0.419016,0.835787,site,Not Significant,0.419016,0.835787
1,time,Not Significant,3.271963,0.070590,time,Not Significant,3.271963,0.070590
2,treat,*Significant*,10.159258,0.000001,treat,*Significant*,10.159258,0.000001
3,site_treat,Not Significant,1.396898,0.139362,site_treat,Not Significant,1.396898,0.139362
4,time_treat,Not Significant,1.177557,0.316772,time_treat,Not Significant,1.177557,0.316772
5,site_time_treat,Not Significant,1.245526,0.250721,site_time_treat,Not Significant,1.245526,0.250721


### SSRS internalizing 

In [ ]:
outcome = dict_outcome_var['ssrs_int']
outcome

'sspintx'

### 14 months 

In [ ]:
res_ssrs_14_p, summ_ssrs_14_p , h = rr.get_RR_stats(formulas[outcome] , ssrs14_p, group, alpha)
res_ssrs_14_t, summ_ssrs_14_t , h2 = rr.get_RR_stats(formulas[outcome], ssrs14_t, group,  alpha)
f_test_ssrs14_p = rr.f_test_interactions(res_ssrs_14_p, hyps_interactions, alpha)
f_test_ssrs14_t = rr.f_test_interactions(res_ssrs_14_t, hyps_interactions, alpha)

f_test_ssrs14 = pd.concat([f_test_ssrs14_p, f_test_ssrs14_p], axis=1)
f_test_ssrs14

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value,Description,Significance,F-Value,P-Value
0,site,Not Significant,0.630444,6.765408e-01,site,Not Significant,0.630444,6.765408e-01
1,time,Not Significant,1.806427,1.790183e-01,time,Not Significant,1.806427,1.790183e-01
2,treat,*Significant*,11.550932,1.554434e-07,treat,*Significant*,11.550932,1.554434e-07
3,site_treat,Not Significant,0.752609,7.316562e-01,site_treat,Not Significant,0.752609,7.316562e-01
4,time_treat,Not Significant,1.769876,1.506751e-01,time_treat,Not Significant,1.769876,1.506751e-01
5,site_time_treat,Not Significant,0.639782,7.958814e-01,site_time_treat,Not Significant,0.639782,7.958814e-01


### 24 months

In [ ]:
res_ssrs_24_p, summ_ssrs_24_p , h = rr.get_RR_stats(formulas[outcome] , ssrs24_p, group, alpha)
res_ssrs_24_t, summ_ssrs_24_t , h2 = rr.get_RR_stats(formulas[outcome], ssrs24_t, group,  alpha)
f_test_ssrs24_p = rr.f_test_interactions(res_ssrs_24_p, hyps_interactions, alpha)
f_test_snap24_p


f_test_ssrs24_t = rr.f_test_interactions(res_ssrs_24_t, hyps_interactions, alpha)

f_test_ssrs24 = pd.concat([f_test_ssrs24_p, f_test_ssrs24_p], axis=1)
f_test_ssrs24

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value,Description,Significance,F-Value,P-Value
0,site,Not Significant,0.649316,0.662043,site,Not Significant,0.649316,0.662043
1,time,Not Significant,1.752057,0.185685,time,Not Significant,1.752057,0.185685
2,treat,*Significant*,9.473577,0.000003,treat,*Significant*,9.473577,0.000003
3,site_treat,Not Significant,0.892636,0.572282,site_treat,Not Significant,0.892636,0.572282
4,time_treat,Not Significant,0.560491,0.641089,time_treat,Not Significant,0.560491,0.641089
5,site_time_treat,Not Significant,0.922627,0.517161,site_time_treat,Not Significant,0.922627,0.517161


In [ ]:


#.style#.applymap(
    #lambda v: highlight_significant_p_values(v, alpha), 
    #subset=['P-Value'])

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value,Description,Significance,F-Value,P-Value
0,site,Not Significant,0.649316,0.662043,site,Not Significant,0.649316,0.662043
1,time,Not Significant,1.752057,0.185685,time,Not Significant,1.752057,0.185685
2,treat,*Significant*,9.473577,0.000003,treat,*Significant*,9.473577,0.000003
3,site_treat,Not Significant,0.892636,0.572282,site_treat,Not Significant,0.892636,0.572282
4,time_treat,Not Significant,0.560491,0.641089,time_treat,Not Significant,0.560491,0.641089
5,site_time_treat,Not Significant,0.922627,0.517161,site_time_treat,Not Significant,0.922627,0.517161


### 36 months 

In [ ]:
res_ssrs_36_p, summ_ssrs_24_p , h = rr.get_RR_stats(formulas[outcome] , ssrs36_p, group, alpha)
res_ssrs_36_t, summ_ssrs_24_t , h2 = rr.get_RR_stats(formulas[outcome], ssrs36_t, group,  alpha)
f_test_ssrs36_p = rr.f_test_interactions(res_ssrs_36_p, hyps_interactions, alpha)

f_test_ssrs36_t = rr.f_test_interactions(res_ssrs_36_t, hyps_interactions, alpha)

f_test_ssrs36 = pd.concat([f_test_ssrs36_p, f_test_ssrs36_p], axis=1)
f_test_ssrs36


/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value,Description,Significance,F-Value,P-Value
0,site,Not Significant,0.631128,0.676013,site,Not Significant,0.631128,0.676013
1,time,Not Significant,1.743970,0.186699,time,Not Significant,1.743970,0.186699
2,treat,*Significant*,9.925533,0.000002,treat,*Significant*,9.925533,0.000002
3,site_treat,Not Significant,0.842389,0.630366,site_treat,Not Significant,0.842389,0.630366
4,time_treat,Not Significant,0.463421,0.707832,time_treat,Not Significant,0.463421,0.707832
5,site_time_treat,Not Significant,0.680733,0.758195,site_time_treat,Not Significant,0.680733,0.758195


## SSRS social skills 

In [ ]:
outcome = dict_outcome_var['ssrs_ss']
outcome

'ssptossx'

In [ ]:
res_ssrs_14_p, summ_ssrs_14_p , h = rr.get_RR_stats(formulas[outcome] , ssrs14_p, group, alpha)
res_ssrs_14_t, summ_ssrs_14_t , h2 = rr.get_RR_stats(formulas[outcome], ssrs14_t, group,  alpha)
f_test_ssrs14_p = rr.f_test_interactions(res_ssrs_14_p, hyps_interactions, alpha)
f_test_ssrs14_t = rr.f_test_interactions(res_ssrs_14_t, hyps_interactions, alpha)

f_test_ssrs14 = pd.concat([f_test_ssrs14_p, f_test_ssrs14_p], axis=1)
f_test_ssrs14

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value,Description,Significance,F-Value,P-Value
0,site,Not Significant,0.968414,4.356056e-01,site,Not Significant,0.968414,4.356056e-01
1,time,Not Significant,1.376538,2.407659e-01,time,Not Significant,1.376538,2.407659e-01
2,treat,*Significant*,24.045043,2.075020e-15,treat,*Significant*,24.045043,2.075020e-15
3,site_treat,Not Significant,0.924148,5.361241e-01,site_treat,Not Significant,0.924148,5.361241e-01
4,time_treat,Not Significant,0.877618,4.518577e-01,time_treat,Not Significant,0.877618,4.518577e-01
5,site_time_treat,Not Significant,0.505155,9.010972e-01,site_time_treat,Not Significant,0.505155,9.010972e-01


In [ ]:
res_ssrs_24_p, summ_ssrs_24_p , h = rr.get_RR_stats(formulas[outcome] , ssrs24_p, group, alpha)
res_ssrs_24_t, summ_ssrs_24_t , h2 = rr.get_RR_stats(formulas[outcome], ssrs24_t, group,  alpha)
f_test_ssrs24_p = rr.f_test_interactions(res_ssrs_24_p, hyps_interactions, alpha)
f_test_snap24_p


f_test_ssrs24_t = rr.f_test_interactions(res_ssrs_24_t, hyps_interactions, alpha)

f_test_ssrs24 = pd.concat([f_test_ssrs24_p, f_test_ssrs24_p], axis=1)
f_test_ssrs24

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value,Description,Significance,F-Value,P-Value
0,site,Not Significant,0.913316,4.711298e-01,site,Not Significant,0.913316,4.711298e-01
1,time,Not Significant,1.359860,2.436232e-01,time,Not Significant,1.359860,2.436232e-01
2,treat,*Significant*,22.882123,1.069057e-14,treat,*Significant*,22.882123,1.069057e-14
3,site_treat,Not Significant,0.991045,4.613579e-01,site_treat,Not Significant,0.991045,4.613579e-01
4,time_treat,Not Significant,0.369419,7.750841e-01,time_treat,Not Significant,0.369419,7.750841e-01
5,site_time_treat,Not Significant,0.813178,6.269683e-01,site_time_treat,Not Significant,0.813178,6.269683e-01


In [ ]:
res_ssrs_36_p, summ_ssrs_24_p , h = rr.get_RR_stats(formulas[outcome] , ssrs36_p, group, alpha)
res_ssrs_36_t, summ_ssrs_24_t , h2 = rr.get_RR_stats(formulas[outcome], ssrs36_t, group,  alpha)
f_test_ssrs36_p = rr.f_test_interactions(res_ssrs_36_p, hyps_interactions, alpha)

f_test_ssrs36_t = rr.f_test_interactions(res_ssrs_36_t, hyps_interactions, alpha)

f_test_ssrs36 = pd.concat([f_test_ssrs36_p, f_test_ssrs36_p], axis=1)
f_test_ssrs36

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value,Description,Significance,F-Value,P-Value
0,site,Not Significant,0.905715,4.761650e-01,site,Not Significant,0.905715,4.761650e-01
1,time,Not Significant,1.336198,2.477629e-01,time,Not Significant,1.336198,2.477629e-01
2,treat,*Significant*,22.290127,2.494658e-14,treat,*Significant*,22.290127,2.494658e-14
3,site_treat,Not Significant,1.020275,4.299980e-01,site_treat,Not Significant,1.020275,4.299980e-01
4,time_treat,Not Significant,0.314686,8.147834e-01,time_treat,Not Significant,0.314686,8.147834e-01
5,site_time_treat,Not Significant,0.986514,4.559503e-01,site_time_treat,Not Significant,0.986514,4.559503e-01


### MASC 

In [ ]:
outcome = dict_outcome_var['masc_tot']
outcome

'masc_masctotalt'

In [ ]:
res_masc14, summ_masc_14 , h = rr.get_RR_stats(formulas[outcome] , masc14, group, alpha)
f_test_masc14 = rr.f_test_interactions(res_masc14, hyps_interactions, alpha)
f_test_masc14

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value
0,site,Not Significant,0.838696,0.522074
1,time,Not Significant,2.891338,0.089203
2,treat,Not Significant,0.992599,0.395276
3,site_treat,*Significant*,1.956234,0.015014
4,time_treat,Not Significant,1.451131,0.226070
5,site_time_treat,*Significant*,2.107876,0.017055


In [ ]:
res_masc24, summ_masc_14 , h = rr.get_RR_stats(formulas[outcome] , masc24, group, alpha)
f_test_masc24 = rr.f_test_interactions(res_masc24, hyps_interactions, alpha)
f_test_masc24

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value
0,site,Not Significant,0.897563,0.481683
1,time,Not Significant,3.172017,0.075024
2,treat,Not Significant,0.794508,0.496825
3,site_treat,*Significant*,1.683452,0.047399
4,time_treat,Not Significant,1.679463,0.169284
5,site_time_treat,*Significant*,2.396302,0.005908


In [ ]:
res_masc36, summ_masc_14 , h = rr.get_RR_stats(formulas[outcome] , masc36, group, alpha)
f_test_masc36 = rr.f_test_interactions(res_masc36, hyps_interactions, alpha)
f_test_masc36

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value
0,site,Not Significant,1.026292,0.400271
1,time,Not Significant,3.325667,0.068309
2,treat,Not Significant,1.158213,0.324259
3,site_treat,Not Significant,1.386815,0.144190
4,time_treat,Not Significant,1.389386,0.244099
5,site_time_treat,*Significant*,2.108632,0.016895


## Wechsler reading 

In [ ]:
outcome = dict_outcome_var['w_read']
outcome

'w1readb'

In [ ]:
res_w14, summ_w_14 , h = rr.get_RR_stats(formulas[outcome] , wechsler14, group, alpha)
f_test_w14= rr.f_test_interactions(res_w14, hyps_interactions, alpha)
f_test_w14

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value
0,site,*Significant*,2.736320,0.018097
1,time,Not Significant,0.778405,0.377763
2,treat,*Significant*,2.672961,0.045988
3,site_treat,Not Significant,0.935301,0.523636
4,time_treat,Not Significant,0.635075,0.592424
5,site_time_treat,Not Significant,0.733597,0.706847


In [ ]:
res_w24, summ_w_14 , h = rr.get_RR_stats(formulas[outcome] , wechsler24, group, alpha)
f_test_w24= rr.f_test_interactions(res_w24, hyps_interactions, alpha)
f_test_w24

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value
0,site,*Significant*,2.752376,0.017440
1,time,Not Significant,0.805567,0.369536
2,treat,*Significant*,2.996608,0.029654
3,site_treat,Not Significant,0.941781,0.516225
4,time_treat,Not Significant,0.741996,0.526974
5,site_time_treat,Not Significant,0.947769,0.492878


In [ ]:
res_w36, summ_w_36 , h = rr.get_RR_stats(formulas[outcome] , wechsler36, group, alpha)
f_test_w36= rr.f_test_interactions(res_w36, hyps_interactions, alpha)
f_test_w36

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value
0,site,*Significant*,2.723156,0.018468
1,time,Not Significant,0.824727,0.363894
2,treat,*Significant*,3.008563,0.029151
3,site_treat,Not Significant,0.961196,0.494415
4,time_treat,Not Significant,0.887917,0.446582
5,site_time_treat,Not Significant,1.127396,0.334872


## Wechsler math 

In [ ]:
outcome = dict_outcome_var['w_math']
outcome

'w2math'

In [ ]:
res_w14, summ_w_14 , h = rr.get_RR_stats(formulas[outcome] , wechsler14, group, alpha)
f_test_w14= rr.f_test_interactions(res_w14, hyps_interactions, alpha)
f_test_w14

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value
0,site,Not Significant,1.924660,0.087290
1,time,Not Significant,0.000090,0.992446
2,treat,*Significant*,4.589047,0.003325
3,site_treat,Not Significant,1.141942,0.312511
4,time_treat,Not Significant,0.376829,0.769732
5,site_time_treat,Not Significant,1.361454,0.184894


In [ ]:
res_w24, summ_w_14 , h = rr.get_RR_stats(formulas[outcome] , wechsler24, group, alpha)
f_test_w24= rr.f_test_interactions(res_w24, hyps_interactions, alpha)
f_test_w24

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value
0,site,Not Significant,1.814101,0.106792
1,time,Not Significant,0.000143,0.990454
2,treat,*Significant*,4.426135,0.004145
3,site_treat,Not Significant,1.047016,0.402590
4,time_treat,Not Significant,0.368867,0.775487
5,site_time_treat,Not Significant,0.894697,0.544861


In [ ]:
res_w36, summ_w_36 , h = rr.get_RR_stats(formulas[outcome] , wechsler36, group, alpha)
f_test_w36= rr.f_test_interactions(res_w36, hyps_interactions, alpha)
f_test_w36

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value
0,site,Not Significant,1.739620,0.122187
1,time,Not Significant,0.000129,0.990939
2,treat,*Significant*,4.583750,0.003321
3,site_treat,Not Significant,1.065005,0.384372
4,time_treat,Not Significant,1.237425,0.294510
5,site_time_treat,*Significant*,1.899293,0.035062


In [ ]:
wechsler14.keys()

Index(['src_subject_id', 'interview_date', 'interview_age', 'sex', 'site',
       'days_baseline', 'relationship', 'w1readb', 'w2math', 'w3spell',
       'log_days_baseline', 'trtname'],
      dtype='object')

In [ ]:
outcome = dict_outcome_var['w_spell']
outcome

'w3spell'

In [ ]:
res_w14, summ_w_14 , h = rr.get_RR_stats(formulas[outcome] , wechsler14, group, alpha)
f_test_w14= rr.f_test_interactions(res_w14, hyps_interactions, alpha)
f_test_w14

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value
0,site,Not Significant,1.516170,0.181586
1,time,Not Significant,0.580384,0.446275
2,treat,*Significant*,2.858653,0.035823
3,site_treat,Not Significant,0.609549,0.869312
4,time_treat,Not Significant,0.060611,0.980460
5,site_time_treat,*Significant*,1.862997,0.039912


In [ ]:
res_w24, summ_w_14 , h = rr.get_RR_stats(formulas[outcome] , wechsler24, group, alpha)
f_test_w24= rr.f_test_interactions(res_w24, hyps_interactions, alpha)
f_test_w24

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value
0,site,Not Significant,1.434141,0.208752
1,time,Not Significant,0.596781,0.439895
2,treat,*Significant*,2.877067,0.034862
3,site_treat,Not Significant,0.602544,0.875068
4,time_treat,Not Significant,0.260789,0.853670
5,site_time_treat,Not Significant,1.316008,0.208708


In [ ]:
res_w36, summ_w_36 , h = rr.get_RR_stats(formulas[outcome] , wechsler36, group, alpha)
f_test_w36= rr.f_test_interactions(res_w36, hyps_interactions, alpha)
f_test_w36

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


,Description,Significance,F-Value,P-Value
0,site,Not Significant,1.340149,0.244285
1,time,Not Significant,0.594926,0.440597
2,treat,*Significant*,2.651998,0.047175
3,site_treat,Not Significant,0.640845,0.842969
4,time_treat,Not Significant,0.738707,0.528891
5,site_time_treat,Not Significant,1.311586,0.211075
